In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"  # Suppress Hugging Face symlink warning

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Neo4jVector
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_groq import ChatGroq
import tempfile
from IPython.display import display, clear_output
import ipywidgets as widgets

In [ ]:
def connect_to_neo4j():
    print("Connecting to Neo4j Database...")
    neo4j_url = input("Enter Neo4j URL (e.g., neo4j+s://<your-neo4j-url>): ")
    neo4j_username = input("Enter Neo4j Username (default: neo4j): ") or "neo4j"
    neo4j_password = input("Enter Neo4j Password: ")

    try:
        graph = Neo4jGraph(
            url=neo4j_url,
            username=neo4j_username,
            password=neo4j_password
        )
        print("Successfully connected to Neo4j database!")
        return graph, neo4j_url, neo4j_username, neo4j_password
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")
        return None, None, None, None

In [ ]:
def process_pdf(file_path, graph, neo4j_url, neo4j_username, neo4j_password, embeddings, llm):
    print("Processing PDF...")

    # Load and split the PDF
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
    docs = text_splitter.split_documents(pages)

    lc_docs = []
    for doc in docs:
        lc_docs.append(Document(
            page_content=doc.page_content.replace("\n", ""),
            metadata={'source': file_path.split('/')[-1]}
        ))

    # Clear the graph database
    cypher = """
    MATCH (n)
    DETACH DELETE n;
    """
    graph.query(cypher)

    # Define allowed nodes and relationships
    allowed_nodes = ["Disease", "Cause", "Symptom", "Treatment", "Herb", "AsthmaType", "Season"]
    allowed_relationships = ["HAS_TYPE", "CAUSED_BY", "HAS_SYMPTOM", "TREATED_BY",
                           "ASSOCIATED_WITH", "MODIFIED_BY", "HAS_SEASONALITY", "HAS_CAUSE"]

    # Transform documents into graph documents
    transformer = LLMGraphTransformer(
        llm=llm,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
        node_properties=False,
        relationship_properties=False
    )

    graph_documents = transformer.convert_to_graph_documents(lc_docs)
    graph.add_graph_documents(graph_documents, include_source=True)

    # Create vector index
    index = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        url=neo4j_url,
        username=neo4j_username,
        password=neo4j_password,
        database="neo4j",
        node_label="Disease",
        text_node_properties=["id", "text"],
        embedding_node_property="embedding",
        index_name="vector_index",
        keyword_index_name="entity_index",
        search_type="hybrid"
    )

    print("PDF processing complete!")
    return index

In [ ]:
def setup_qa_chain(graph, llm):
    schema = graph.get_schema

    template = """
    Task: Generate a Cypher statement to query the graph database.

    Instructions:
    - Use the exact relationship types and node labels from the schema.
    - Return only relevant node properties.

    Schema:
    {schema}

    Question: {question}"""

    question_prompt = PromptTemplate(
        template=template,
        input_variables=["schema", "question"]
    )

    qa = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=question_prompt,
        verbose=True,
        allow_dangerous_requests=True
    )
    return qa

In [ ]:
def main():
    # Set embeddings
    embeddings = HuggingFaceEmbeddings(model_name='all-mpnet-base-v2')

    # Initialize Groq LLM
    groq_api_key = ''  # Replace with your actual key
    llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama3-70b-8192",
        temperature=0.2
    )

    # Connect to Neo4j
    graph, neo4j_url, neo4j_username, neo4j_password = connect_to_neo4j()
    if graph is None:
        return

    # File upload in Colab
    from google.colab import files
    uploaded = files.upload()

    if uploaded:
        for filename, file_content in uploaded.items():
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(file_content)
                tmp_file_path = tmp_file.name

            # Process the PDF
            index = process_pdf(tmp_file_path, graph, neo4j_url, neo4j_username, neo4j_password, embeddings, llm)

            # Setup QA chain
            qa = setup_qa_chain(graph, llm)

            # Question input loop
            while True:
                question = input("\nEnter your question (or 'quit' to exit): ")
                if question.lower() == 'quit':
                    break
                print("Generating answer...")
                res = qa.invoke({"query": question})
                print("\nAnswer:", res['result'])

if __name__ == "__main__":
    main()

Connecting to Neo4j Database...
Enter Neo4j URL (e.g., neo4j+s://<your-neo4j-url>): neo4j+s://df9f2ca8.databases.neo4j.io
Enter Neo4j Username (default: neo4j): neo4j
Enter Neo4j Password: P5A0PfuhF9z1misKFuDxbRRuijR7F-DPOAS_hf-G7t4
Successfully connected to Neo4j database!


Saving Chapter 2.pdf to Chapter 2 (1).pdf
Processing PDF...


In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq

# Set up your Neo4j credentials
NEO4J_URL = "neo4j+s://df9f2ca8.databases.neo4j.io"  # Replace with your actual Neo4j URL
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "P5A0PfuhF9z1misKFuDxbRRuijR7F-DPOAS_hf-G7t4"  # Replace with your actual password

# Initialize Neo4j graph connection
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Set up Groq LLM API (Replace with your actual API key)
GROQ_API_KEY = "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"

llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192",
    temperature=0.2
)

# Retrieve Neo4j Schema
schema = graph.get_schema

# Define the prompt template for generating Cypher queries
template = """
Task: Generate Cypher to query the Ayurvedic knowledge graph.
Use ONLY these node labels and relationships:
- Nodes: Disease, Cause, Symptom, Treatment, Prevention, Prohibited, Diet, Factor, Stage, Trigger
- Relationships: CAUSES, HAS_SYMPTOM, TREATED_BY, PREVENTS, PROHIBITED, RECOMMENDED_FOR, AGGRAVATES, STAGE_OF, TRIGGERED_BY

Rules:
- Do not use multiple RETURN statements separately. Use UNION if you need to return different types of data.
- Use RETURN statements only at the end of the query.

Schema:
{schema}

Question: {question}
"""


question_prompt = PromptTemplate(
    template=template,
    input_variables=["schema", "question"]
)

# Set up the QA Chain
qa = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=question_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

# **Ask Questions in a Loop**
print("💡 Ask a question about Ayurveda (type 'exit' to quit):")
while True:
    question = input("📝 Your Question: ")
    if question.lower() == "exit":
        print("👋 Exiting...")
        break

    # Generate and execute Cypher query
    response = qa.invoke({"query": question})

    print("\n🔍 **Answer:**", response['result'], "\n")


<ipython-input-7-1ea4cb74f945>:13: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


💡 Ask a question about Ayurveda (type 'exit' to quit):
📝 Your Question: What are the causes of Varicose Veins?


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (d:Disease {name: "Varicose Veins"})<-[:CAUSES]-(c:Cause)
RETURN c.name AS Cause, c.description AS Description

Full Context:
[{'Cause': 'Weak Nervous System', 'Description': 'Overactive nerves cause veins to dilate abnormally and lose elasticity.'}, {'Cause': 'Loss of Blood', 'Description': 'Weakens the nervous system, making veins more susceptible to dilation.'}, {'Cause': 'Blockage in Circulation', 'Description': 'Poor blood flow prevents veins from returning to normal size.'}, {'Cause': 'Weak Digestion and Malnutrition', 'Description': 'Lack of nutrients weakens vein walls and nervous function.'}, {'Cause': 'Pregnancy', 'Description': 'Expanded uterus puts pressure on iliac veins, causing circulation issues.'}, {'Cause': 'Menopause and Heavy Menstruation', 'Description': 'Hormonal changes and blood loss 